In [1]:
#dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

#import api key
from config import weather_api_key

#incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

#output file (csv)
output_data_file = "output_data/cities.csv"

#range of latitudes and longitudes
#lat_range = (-90, 90)
#lng_range = (-180, 180)
lat_range = (-90, -18)
lng_range = (-180, -150)

#formatting for bold text
class color:
   BOLD = '\033[1m'
   END = '\033[0m'

In [ ]:
#printing out the minneapolis json to see the data
url = "http://api.openweathermap.org/data/2.5/weather?"
query_url = f"{url}appid={weather_api_key}&q="
response = requests.get(query_url + "minneapolis").json()
response

In [2]:
#List for holding lat_lngs and cities
lat_lngs = []
cities = []

#create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=-18.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=-150.000, size=1500)
lat_lngs = zip(lats, lngs)

# Add a one second interval between queries to stay within API query limits
time.sleep(1)

#identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

#print the city count to confirm sufficient count
len(cities)

10

In [7]:
#perform a weather check on each city using a series of successive API calls.
#include a print log of each city as it's being processed (with the city number and city name).

#base url
url = "http://api.openweathermap.org/data/2.5/weather?"
#build partial query url
query_url = f"{url}appid={weather_api_key}&q="

#create lists to store city info that I'm interested in
city_names = []
cloudiness = []
country = []
date = []
humidity = []
city_lats = []
city_lngs = []
max_temp = []
wind_speed = []
city_numbers = []

#trying to store data in this list in order to create a dataframe as I loop through (???)
city_info = []

#in order to print the print log according to how the homework looks (change comment before final draft)
i = 1
set_number = 1



print("Beginning Data Retrieval") 
print("--------------------------------")

for j, city in enumerate(cities):
    
    if (j % 50 == 0 and j >= 50):
        set_number = set_number + 1
        i = 0
    print(f"Processing Record {i} of Set {set_number} | {city}")
    i=i+1
   
    response = requests.get(query_url + city).json()
    
    # Add a one second interval between queries to stay within API query limits
    time.sleep(1)
    
    try:
        city_names.append(city)
        city_numbers.append(response['id'])
        city_lats.append(response['coord']['lat'])
        cloudiness.append(response['clouds']['all'])
        country.append(response['sys']['country'])
        date.append(response['dt'])
        humidity.append(response['main']['humidity'])
        city_lngs.append(response['coord']['lon'])
        max_temp.append(response['main']['temp_max'])
        wind_speed.append(response['wind']['speed'])
        city_info.append({"City": [city_names],
                "Cloudiness": cloudiness,
                "Country": country,
                "Date": date,
                "Humidity": humidity,
                "Lat": city_lats,
                "Lng": city_lngs,
                "Max Temp": max_temp,
                "Wind Speed": wind_speed})
      
    except:
        print("City " + color.BOLD + "not " + color.END + "found. Skipping...")
        pass

print("------------------------")
print("Data Retrieval Complete")
print("------------------------")

#trying to see what I've created, what I'm working with, what's going wrong!!!
print(city_info)
type(city_info)

Beginning Data Retrieval
--------------------------------
Processing Record 1 of Set 1 | avarua
Processing Record 2 of Set 1 | mataura
Processing Record 3 of Set 1 | vaini
Processing Record 4 of Set 1 | alofi
Processing Record 5 of Set 1 | avera
Processing Record 6 of Set 1 | moerai
Processing Record 7 of Set 1 | pangai
Processing Record 8 of Set 1 | neiafu
Processing Record 9 of Set 1 | tevaitoa
Processing Record 10 of Set 1 | haapiti
------------------------
Data Retrieval Complete
------------------------
[{'City': [['avarua', 'mataura', 'vaini', 'alofi', 'avera', 'moerai', 'pangai', 'neiafu', 'tevaitoa', 'haapiti']], 'Cloudiness': [100, 100, 20, 96, 1, 100, 20, 20, 88, 20], 'Country': ['CK', 'NZ', 'TO', 'NU', 'US', 'PF', 'TO', 'TO', 'PF', 'PF'], 'Date': [1579556228, 1579556229, 1579556230, 1579556231, 1579556232, 1579556233, 1579556234, 1579556236, 1579556237, 1579556238], 'Humidity': [83, 88, 65, 72, 36, 83, 58, 61, 77, 70], 'Lat': [-21.21, -46.19, -21.2, -19.06, 33.19, -22.43, -1

list

In [4]:
#make a dataframe that includes the info that I've just found
#all the columns are correct, but obviously there are lists inside lists?
city_df = pd.DataFrame(city_info)
city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,"[[avarua, mataura, vaini, alofi, avera, moerai...","[100, 100, 20, 96, 1, 100, 20, 20, 88, 20]","[CK, NZ, TO, NU, US, PF, TO, TO, PF, PF]","[1579555217, 1579555219, 1579555220, 157955522...","[83, 90, 65, 72, 36, 83, 58, 61, 77, 70]","[-21.21, -46.19, -21.2, -19.06, 33.19, -22.43,...","[-159.78, 168.86, -175.2, -169.92, -82.53, -15...","[301.15, 288.71, 301.15, 299, 281.15, 299.34, ...","[5.1, 1.79, 5.7, 4.28, 4.1, 7.06, 3.1, 2.6, 6...."
1,"[[avarua, mataura, vaini, alofi, avera, moerai...","[100, 100, 20, 96, 1, 100, 20, 20, 88, 20]","[CK, NZ, TO, NU, US, PF, TO, TO, PF, PF]","[1579555217, 1579555219, 1579555220, 157955522...","[83, 90, 65, 72, 36, 83, 58, 61, 77, 70]","[-21.21, -46.19, -21.2, -19.06, 33.19, -22.43,...","[-159.78, 168.86, -175.2, -169.92, -82.53, -15...","[301.15, 288.71, 301.15, 299, 281.15, 299.34, ...","[5.1, 1.79, 5.7, 4.28, 4.1, 7.06, 3.1, 2.6, 6...."
2,"[[avarua, mataura, vaini, alofi, avera, moerai...","[100, 100, 20, 96, 1, 100, 20, 20, 88, 20]","[CK, NZ, TO, NU, US, PF, TO, TO, PF, PF]","[1579555217, 1579555219, 1579555220, 157955522...","[83, 90, 65, 72, 36, 83, 58, 61, 77, 70]","[-21.21, -46.19, -21.2, -19.06, 33.19, -22.43,...","[-159.78, 168.86, -175.2, -169.92, -82.53, -15...","[301.15, 288.71, 301.15, 299, 281.15, 299.34, ...","[5.1, 1.79, 5.7, 4.28, 4.1, 7.06, 3.1, 2.6, 6...."
3,"[[avarua, mataura, vaini, alofi, avera, moerai...","[100, 100, 20, 96, 1, 100, 20, 20, 88, 20]","[CK, NZ, TO, NU, US, PF, TO, TO, PF, PF]","[1579555217, 1579555219, 1579555220, 157955522...","[83, 90, 65, 72, 36, 83, 58, 61, 77, 70]","[-21.21, -46.19, -21.2, -19.06, 33.19, -22.43,...","[-159.78, 168.86, -175.2, -169.92, -82.53, -15...","[301.15, 288.71, 301.15, 299, 281.15, 299.34, ...","[5.1, 1.79, 5.7, 4.28, 4.1, 7.06, 3.1, 2.6, 6...."
4,"[[avarua, mataura, vaini, alofi, avera, moerai...","[100, 100, 20, 96, 1, 100, 20, 20, 88, 20]","[CK, NZ, TO, NU, US, PF, TO, TO, PF, PF]","[1579555217, 1579555219, 1579555220, 157955522...","[83, 90, 65, 72, 36, 83, 58, 61, 77, 70]","[-21.21, -46.19, -21.2, -19.06, 33.19, -22.43,...","[-159.78, 168.86, -175.2, -169.92, -82.53, -15...","[301.15, 288.71, 301.15, 299, 281.15, 299.34, ...","[5.1, 1.79, 5.7, 4.28, 4.1, 7.06, 3.1, 2.6, 6...."
5,"[[avarua, mataura, vaini, alofi, avera, moerai...","[100, 100, 20, 96, 1, 100, 20, 20, 88, 20]","[CK, NZ, TO, NU, US, PF, TO, TO, PF, PF]","[1579555217, 1579555219, 1579555220, 157955522...","[83, 90, 65, 72, 36, 83, 58, 61, 77, 70]","[-21.21, -46.19, -21.2, -19.06, 33.19, -22.43,...","[-159.78, 168.86, -175.2, -169.92, -82.53, -15...","[301.15, 288.71, 301.15, 299, 281.15, 299.34, ...","[5.1, 1.79, 5.7, 4.28, 4.1, 7.06, 3.1, 2.6, 6...."
6,"[[avarua, mataura, vaini, alofi, avera, moerai...","[100, 100, 20, 96, 1, 100, 20, 20, 88, 20]","[CK, NZ, TO, NU, US, PF, TO, TO, PF, PF]","[1579555217, 1579555219, 1579555220, 157955522...","[83, 90, 65, 72, 36, 83, 58, 61, 77, 70]","[-21.21, -46.19, -21.2, -19.06, 33.19, -22.43,...","[-159.78, 168.86, -175.2, -169.92, -82.53, -15...","[301.15, 288.71, 301.15, 299, 281.15, 299.34, ...","[5.1, 1.79, 5.7, 4.28, 4.1, 7.06, 3.1, 2.6, 6...."
7,"[[avarua, mataura, vaini, alofi, avera, moerai...","[100, 100, 20, 96, 1, 100, 20, 20, 88, 20]","[CK, NZ, TO, NU, US, PF, TO, TO, PF, PF]","[1579555217, 1579555219, 1579555220, 157955522...","[83, 90, 65, 72, 36, 83, 58, 61, 77, 70]","[-21.21, -46.19, -21.2, -19.06, 33.19, -22.43,...","[-159.78, 168.86, -175.2, -169.92, -82.53, -15...","[301.15, 288.71, 301.15, 299, 281.15, 299.34, ...","[5.1, 1.79, 5.7, 4.28, 4.1, 7.06, 3.1, 2.6, 6...."
8,"[[avarua, mataura, vaini, alofi, avera, moerai...","[100, 100, 20, 96, 1, 100, 20, 20, 88, 20]","[CK, NZ, TO, NU, US, PF, TO, TO, PF, PF]","[1579555217, 1579555219, 1579555220, 157955522...","[83, 90, 65, 72, 36, 83, 58, 61, 77, 70]","[-21.21, -46.19, -21.2, -19.06, 33.19, -22.43,...","[-159.78, 168.86, -175.2, -169.92, -82.53, -15...","[301.15, 288.71, 301.15, 299, 2

In [5]:
#try creating a dataframe another way
#looks to work, though temps are incorrect
weather_dict = {"City": city_names,
                "Cloudiness": cloudiness,
                "Country": country,
                "Date": date,
                "Humidity": humidity,
                "Lat": city_lats,
                "Lng": city_lngs,
                "Max Temp": max_temp,
                "Wind Speed": wind_speed}
weather_data = pd.DataFrame(weather_dict)
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,avarua,100,CK,1579555217,83,-21.21,-159.78,301.15,5.10
1,mataura,100,NZ,1579555219,90,-46.19,168.86,288.71,1.79
2,vaini,20,TO,1579555220,65,-21.20,-175.20,301.15,5.70
3,alofi,96,NU,1579555221,72,-19.06,-169.92,299.00,4.28
4,avera,1,US,1579555222,36,33.19,-82.53,281.15,4.10
5,moerai,100,PF,1579555223,83,-22.43,-151.33,299.34,7.06
6,pangai,20,TO,1579555225,58,-19.80,-174.35,301.15,3.10
7,neiafu,20,TO,1579555226,61,-18.65,-173.98,300.15,2.60
8,tevaitoa,88,PF,1579555227,77,-16.78,-151.50,300.53,6.29
9,haapiti,20,PF,1579555228,70,-17.57,-149.87,304.15,7.70


In [6]:
#export city data into a csv file
#make into a dataframe

city_df.to_csv(output_data_file, index_label="City_ID")
#weather_data.to_csv(output_data_file, index_label="City_ID")
#census_pd.to_csv("census_data_states.csv", encoding="utf-8", index=False)



FileNotFoundError: [Errno 2] No such file or directory: 'output_data/cities.csv'